In [ ]:
url = 'https://anaconda.org/conda-forge/libta-lib/0.4.0/download/linux-64/libta-lib-0.4.0-h516909a_0.tar.bz2'
!curl -L $url | tar xj -C /usr/lib/x86_64-linux-gnu/ lib --strip-components=1
url = 'https://anaconda.org/conda-forge/ta-lib/0.4.19/download/linux-64/ta-lib-0.4.19-py37ha21ca33_2.tar.bz2'
!curl -L $url | tar xj -C /usr/local/lib/python3.7/dist-packages/ lib/python3.7/site-packages/talib --strip-components=3

In [ ]:
!pip install python-binance
!pip install binance-client

In [3]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go

In [4]:
from get_data import crypto_data

In [5]:
df = crypto_data()

## Prepare dataset

In [6]:
example = df["df_BTC"]
df_input = pd.DataFrame(columns = example.columns)
for single in df.values():
  df_input = df_input.append(single)

In [7]:
df_output = np.zeros(df_input.shape[0])
for i in range(1, df_input.price.shape[0]):
  diff = df_input.price.iloc[i] - df_input.price.iloc[i-1]
  if diff > 0:
    df_output[i] = 1
  elif diff < 0:
    df_output[i] = -1

In [8]:
df_input_test = df_input.iloc[::5]
df_input_train = df_input.iloc[1::5]
df_input_train = df_input_train.append(df_input.iloc[2::5])
df_input_train = df_input_train.append(df_input.iloc[3::5])
df_input_train = df_input_train.append(df_input.iloc[4::5])

In [9]:
df_output_test = df_output[::5]
df_output_train = df_output[1::5]
df_output_train = np.concatenate((df_output_train, df_output[2::5]))
df_output_train = np.concatenate((df_output_train, df_output[3::5]))
df_output_train = np.concatenate((df_output_train, df_output[4::5]))

## Decision trees

In [ ]:
from sklearn import tree
clf = tree.DecisionTreeClassifier(max_depth=7)
clf = clf.fit(df_input_train, df_output_train)
print("decision tree score test: ", clf.score(df_input_test, df_output_test))
print("decision tree score train: ", clf.score(df_input_train, df_output_train))

In [ ]:
df_plot = df["df_BTC"]
pred = clf.predict(df_plot)

fig = go.Figure()
fig.add_trace(go.Scatter(x=df_plot.index, y=df_plot.price, mode="lines", name="price"))
fig.add_trace(go.Scatter(x=df_plot[pred==1].index, y=df_plot[pred==1].price, mode="markers", marker=dict(size=4, color="hotpink"), name="pred up"))
fig.add_trace(go.Scatter(x=df_plot[pred==-1].index, y=df_plot[pred==-1].price, mode="markers", marker=dict(size=4, color="blue"), name="pred down"))
fig.add_trace(go.Scatter(x=df_plot[pred==0].index, y=df_plot[pred==0].price, mode="markers", marker=dict(size=4, color="green"), name="dont know"))


fig.show()

## Naive bayes

In [43]:
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB

In [ ]:
gnb = GaussianNB()
gnb.fit(df_input_train, df_output_train)
print("Gaussian NB score test: ", gnb.score(df_input_test, df_output_test))
print("Gaussian NB score train: ",gnb.score(df_input_train, df_output_train))

gnb = MultinomialNB()
gnb.fit(df_input_train, df_output_train)
print("Multinomial NB score test: ", gnb.score(df_input_test, df_output_test))
print("Multinomial NB score train: ", gnb.score(df_input_train, df_output_train))

In [ ]:
#pojebana akcja
np.unique(pred)

In [ ]:
df_plot = df["df_BTC"]
pred = gnb.predict(df_plot)

fig = go.Figure()
fig.add_trace(go.Scatter(x=df_plot.index, y=df_plot.price, mode="lines", name="price"))
fig.add_trace(go.Scatter(x=df_plot[pred==1].index, y=df_plot[pred==1].price, mode="markers", marker=dict(size=4, color="hotpink"), name="pred up"))
fig.add_trace(go.Scatter(x=df_plot[pred==-1].index, y=df_plot[pred==-1].price, mode="markers", marker=dict(size=4, color="blue"), name="pred down"))
fig.add_trace(go.Scatter(x=df_plot[pred==0].index, y=df_plot[pred==0].price, mode="markers", marker=dict(size=4, color="green"), name="dont know"))


fig.show()

## Bayesian network

# TODO

## Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=200, max_depth=20, random_state=0)
clf.fit(df_input_train, df_output_train)
print("random forest score test: ", clf.score(df_input_test, df_output_test))
print("random forest score train: ", clf.score(df_input_train, df_output_train))

In [ ]:
df_plot = df["df_BTC"]
pred = clf.predict(df_plot)

fig = go.Figure()
fig.add_trace(go.Scatter(x=df_plot.index, y=df_plot.price, mode="lines", name="price"))
fig.add_trace(go.Scatter(x=df_plot[pred==1].index, y=df_plot[pred==1].price, mode="markers", marker=dict(size=4, color="hotpink"), name="pred up"))
fig.add_trace(go.Scatter(x=df_plot[pred==-1].index, y=df_plot[pred==-1].price, mode="markers", marker=dict(size=4, color="blue"), name="pred down"))
fig.add_trace(go.Scatter(x=df_plot[pred==0].index, y=df_plot[pred==0].price, mode="markers", marker=dict(size=4, color="green"), name="dont know"))


fig.show()

## Boosting random forest with weghitng with evolutionara algorithm

# TODO

## SUPPORT VECTOR MACHINES

ktoś chce to sie może pobawić paramkami

In [ ]:
from sklearn import svm
#for gamma in []
clf = svm.SVC()
clf.fit(df_input_train, df_output_train)
print("random forest score test: ", clf.score(df_input_test, df_output_test))
print("random forest score train: ", clf.score(df_input_train, df_output_train))

## Neural network

### Classifier same as in previous example

In [52]:
import tensorflow as tf
from tensorflow import keras
#from ten

In [53]:
#model preparation
ds_output_NN_train = tf.one_hot(df_output_train, depth = 2)
ds_output_NN_test = tf.one_hot(df_output_test, depth = 2)

In [ ]:
df_input.shape[1]

In [ ]:
model = tf.keras.models.Sequential([                    
  #tf.keras.layers.Conv1(filters= 2, kernel_size = 3, activation='tanh', use_bias=False),
  #tf.keras.layers.MaxPool2D(),
  #tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(10, use_bias=True, activation="relu", input_shape=(26,)),
  tf.keras.layers.Dense(5, use_bias=True, activation="tanh"),
  tf.keras.layers.Dense(2, use_bias=True, activation="sigmoid")
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(0.0008),
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.BinaryAccuracy()],
)

model.fit(
    df_input_train,
    ds_output_NN_train,
    epochs=10,
    validation_data = (df_input_test, ds_output_NN_test)
)

In [ ]:
#XD
df_plot = df["df_BTC"]
pred = np.argmax(model.predict(df_plot), axis = 1)

fig = go.Figure()
fig.add_trace(go.Scatter(x=df_plot.index, y=df_plot.price, mode="lines", name="price"))
fig.add_trace(go.Scatter(x=df_plot[pred==1].index, y=df_plot[pred==1].price, mode="markers", marker=dict(size=4, color="hotpink"), name="pred up"))
fig.add_trace(go.Scatter(x=df_plot[pred==-1].index, y=df_plot[pred==-1].price, mode="markers", marker=dict(size=4, color="blue"), name="pred down"))
fig.add_trace(go.Scatter(x=df_plot[pred==0].index, y=df_plot[pred==0].price, mode="markers", marker=dict(size=4, color="green"), name="dont know"))


fig.show()